**late submission**
- copy tabnet from version 83 
- solve as a multiclass classification (metric: logloss)
- data augmentation for minor class by cutmix

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold
from torch.nn.modules.loss import _WeightedLoss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils import check_random_state

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor, TabNetClassifier

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
drug = pd.read_csv(DATA_DIR + 'train_drug.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
def make_string(row):
    return str(row[1:].values).replace('[','').replace(']','').replace('\n','').replace(' ','')

targets["target_pair"] = targets.apply(make_string, axis=1)
targetpair_id = dict(enumerate(targets["target_pair"].unique()))
id_targetpair = {y:x for x,y in targetpair_id.items()}
targets["target_pair_num"] = targets["target_pair"].map(id_targetpair)

multiclass_targets = pd.get_dummies(targets["target_pair_num"])

for i in range(len(id_targetpair.keys())):
    if i == 0:
        classid_target = np.array(list((targetpair_id[i]))).reshape(1,-1)      
    else:
        classid_target = np.vstack([classid_target, np.array(list((targetpair_id[i]))).reshape(1,-1)])
        
classid_target = classid_target.astype(int)
class_num = multiclass_targets.shape[1]
        
multiclass_targets.shape, classid_target.shape

((23814, 328), (328, 206))

In [6]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
multiclass_targets = targets["target_pair_num"].copy().reset_index(drop=True)

In [7]:
tmp = multiclass_targets.value_counts().copy()
minor_class = tmp[tmp==1].index
minor_class_index = multiclass_targets[multiclass_targets.isin(minor_class)].index

In [8]:
minor_class

Int64Index([326, 327, 318, 312, 325, 310], dtype='int64')

# feature engineering

In [9]:
#X = train[g_feats+c_feats].copy().values
#select = VarianceThreshold(threshold=0.7)
#X_new = select.fit_transform(X)
#drop_feats = list(np.array(train[g_feats+c_feats].columns)[select.get_support()==False])
#len(drop_feats)

#train.drop(drop_feats, axis=1, inplace=True)
#test.drop(drop_feats, axis=1, inplace=True)

#g_feats = [i for i in train.columns if "g-" in i]
#c_feats = [i for i in train.columns if "c-" in i]

In [10]:
#for i in c_feats + g_feats:
    #ss = preprocessing.QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
#    ss = preprocessing.RobustScaler()
#    ss.fit(train[i].values.reshape(-1,1))
#    train[i] = ss.transform(train[i].values.reshape(-1,1))
#    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [11]:
#c_num = 1
#pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
#pca = PCA(n_components=c_num,random_state=42)
#c_train = pca.fit_transform(train[c_feats])
#c_test = pca.transform(test[c_feats])
#c_train = pd.DataFrame(c_train, columns=pca_c_cols)
#c_test = pd.DataFrame(c_test, columns=pca_c_cols)

#g_num = 10
#pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
#pca = PCA(n_components=g_num, random_state=42)
#g_train = pca.fit_transform(train[g_feats])
#g_test = pca.transform(test[g_feats])
#g_train = pd.DataFrame(g_train, columns=pca_g_cols)
#g_test = pd.DataFrame(g_test, columns=pca_g_cols)

#train = pd.concat([train, c_train],axis=1)
#test = pd.concat([test, c_test],axis=1)
#train = pd.concat([train, g_train],axis=1)
#test = pd.concat([test, g_test],axis=1)

In [12]:
#uc_num = 1
#um = UMAP(n_neighbors=30, n_components=uc_num, random_state=42)
#um_c_cols = ["um-c"+str(i+1) for i in range(uc_num)]#
#uc_train = um.fit_transform(train[c_feats])
#uc_test = um.transform(test[c_feats])
#uc_train = pd.DataFrame(uc_train, columns=um_c_cols)
#uc_test = pd.DataFrame(uc_test, columns=um_c_cols)

#ug_num = 10
#um = UMAP(n_neighbors=30, n_components=ug_num, random_state=42)
#um_g_cols = ["um-g"+str(i+1) for i in range(ug_num)]
#ug_train = um.fit_transform(train[g_feats])
#ug_test = um.transform(test[g_feats])
#ug_train = pd.DataFrame(ug_train, columns=um_g_cols)
#ug_test = pd.DataFrame(ug_test, columns=um_g_cols)

#train = pd.concat([train, uc_train],axis=1)
#test = pd.concat([test, uc_test],axis=1)
#train = pd.concat([train, ug_train],axis=1)
#test = pd.concat([test, ug_test],axis=1)

In [13]:
#n_clusters_g = 15
#n_clusters_c = 5
#def create_cluster(train, test, kind, n_clusters):
#    if kind == "g":
#        train_ = train[pca_g_cols].copy()
#        test_ = test[pca_g_cols].copy()
#    else:
#        train_ = train[pca_c_cols].copy()
#        test_ = test[pca_c_cols].copy()    
#    kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(train_)
#    train[f'clusters_{kind}'] = kmeans.labels_
#    test[f'clusters_{kind}'] = kmeans.predict(test_)
#    train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#    test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#    return train, test
    
#train, test = create_cluster(train, test, kind = 'g', n_clusters = n_clusters_g)
#train, test = create_cluster(train, test, kind = 'c', n_clusters = n_clusters_c)

In [14]:
def fe(df):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})      
    tmp.drop(["sig_id", "cp_type"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 874) (3982, 874)


In [15]:
# data augmentation by cutmix
# https://www.kaggle.com/yxohrxn/moa-cutmix

def cutmix_for_tabular(x, y=None, alpha=1.0, p=1.0, random_state=None):
    x_ = x.copy()
    n, d = x.shape

    if n is not None and random.random() < p:
        random_state = check_random_state(random_state)
        l = random_state.beta(alpha, alpha)
        mask = random_state.choice([False, True], size=d, p=[l, 1.0 - l])
        mask = np.where(mask)[0]
        shuffle = random_state.choice(n, n, replace=False)
        x_[:,mask] = x_[np.ix_(shuffle,mask)]
        
        if y is not None:
            y = l * y + (1.0 - l) * y[shuffle]
        
    # extract minor class
    tmp_index = np.where(np.isin(np.argmax(y, axis=1), minor_class))[0]
    print(tmp_index)
        
    return x_[tmp_index], np.argmax(y[tmp_index],axis=1)

for i in [0,1,2,3,4]:
    if i == 0:
        train_mod, y_mod = cutmix_for_tabular(train.values, 
                                      pd.get_dummies(multiclass_targets).values, alpha=1, p=1, random_state=i)
    else:
        train_tmp, y_tmp = cutmix_for_tabular(train.values, 
                                      pd.get_dummies(multiclass_targets).values, alpha=1, p=1, random_state=i)
        train_mod = np.concatenate([train_mod, train_tmp], axis=0)
        y_mod = np.concatenate([y_mod, y_tmp])
        
train_mod = pd.DataFrame(train_mod, columns = train.columns)
train = pd.concat([train, train_mod], axis=0).reset_index(drop=True)

y_mod = pd.DataFrame(y_mod)
multiclass_targets = pd.concat([multiclass_targets, y_mod]).reset_index(drop=True)

[  599  2348  5784 12816 17718 20062]
[ 6269  6613  9380 10755 13810 14313]
[ 7176  7455  9098 15661 15970 18894]
[  858  9318  9931 18208 19565 20789]
[ 7176  7455  9098 15661 15970 18894]


In [16]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()
    
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()
fn_multiclass_targets = multiclass_targets.copy().to_numpy().reshape(-1,)

In [17]:
fn_train.shape, fn_test.shape, fn_multiclass_targets.shape

((21978, 874), (3982, 874), (21978,))

# modelling

In [18]:
MAX_EPOCH=200
device = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def modelling_tabnet(tr, target, te, sample_seed, target_shape):
    seed_everything(sample_seed) 
    tabnet_params = dict(n_d=8, n_a=8, n_steps=1, gamma=1.3, seed = sample_seed,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []

    oof_preds = np.zeros([len(tr),target_shape])
    scores = []
    NB_SPLITS = 6
    mskf = StratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(tr, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx]
        X_val, y_val = tr[val_idx, :], target[val_idx]
        model = TabNetClassifier(**tabnet_params)
    
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ['logloss'],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1)
    
        preds_val = model.predict_proba(X_val)
        # Apply sigmoid to the predictions
        #preds_val =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logloss"])
        oof_preds[val_idx,:] = preds_val
        scores.append(score)

        # preds on test
        preds_test = model.predict_proba(te)
        test_cv_preds.append(-preds_test)
        
    test_preds_all = np.stack(test_cv_preds)
    return oof_preds, test_preds_all

target_oof = np.zeros([len(fn_train),class_num])
target_pred = np.zeros([len(fn_test),class_num])

seeds = [0,1]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(fn_train, fn_multiclass_targets, fn_test, seed_, class_num)
    target_oof += oof_preds / len(seeds)
    target_pred += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 5.63938 | val_logloss: 5.34741 |  0:00:01s
epoch 10 | loss: 3.29167 | val_logloss: 3.35576 |  0:00:13s
epoch 20 | loss: 3.04215 | val_logloss: 3.25486 |  0:00:24s
epoch 30 | loss: 2.84841 | val_logloss: 3.30825 |  0:00:36s

Early stopping occured at epoch 37 with best_epoch = 17 and best_val_logloss = 3.25334
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 5.62497 | val_logloss: 5.19518 |  0:00:01s
epoch 10 | loss: 3.27392 | val_logloss: 3.29962 |  0:00:12s
epoch 20 | loss: 3.05437 | val_logloss: 3.24507 |  0:00:24s
epoch 30 | loss: 2.84964 | val_logloss: 3.30106 |  0:00:35s

Early stopping occured at epoch 39 with best_epoch = 19 and best_val_logloss = 3.2313
Best weights from best epoch are automatically used!
FOLDS :  3
Device used : cuda
epoch 0  | loss: 5.6524  | val_logloss: 5.31011 |  0:00:01s
epoch 10 | loss: 3.2792  | val_logloss: 3.28837 |  0:00:12s
epoch 20 | loss: 3.04016 | va

In [19]:
target_oof = np.dot(target_oof, classid_target)[:21948,:]
target_pred = np.dot(target_pred, classid_target)

In [20]:
p_min = 0.001
p_max = 1 - p_min

t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.clip(np.array(train_checkscore.iloc[:,1:]),p_min, p_max))))

OOF log loss:  0.015844497285548823
OOF log loss:  0.016106240157341484


In [21]:
train_checkscore.to_csv("tab_newval_oof.csv", index=False)

In [22]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)